In [24]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pandas_market_calendars import get_calendar
from dateutil.relativedelta import relativedelta

In [5]:
polygon_api_key = "KkfCQ7fsZnx0yK4bhX9fD81QplTh0Pf3"
calendar = get_calendar("NYSE")

In [10]:
engine = sqlalchemy.create_engine("sqlite:///C:/Users/lewis/OneDrive/tradingstrategies/databases/momentum_strategy_2_database.db")
tickers = pd.read_sql("SELECT * FROM liquid_stocks", con = engine)["ticker"].values

In [32]:
if datetime.today() != datetime.today().replace(day=1):
    end_date = datetime.today().replace(day=1) - timedelta(days=1)
else:
    end_date = datetime.today() - timedelta(days=1)

start_date = end_date - relativedelta(months=+12) - timedelta(days=1)

dates = calendar.schedule(start_date = start_date, end_date = end_date).index.strftime("%Y-%m-%d").values

In [33]:
stocks_list = pd.DataFrame()
stocks_list['ticker'] = tickers
times = []

In [43]:
for date in dates:
    
    start_time = datetime.now()
    
    stocks_request = pd.json_normalize(requests.get(f"https://api.polygon.io/v2/aggs/grouped/locale/us/market/stocks/{date}?adjusted=true&apiKey={polygon_api_key}").json()["results"])
    stocks_request = stocks_request[stocks_request["T"].isin(tickers)].copy()
    stocks_request = stocks_request.rename(columns={"T":"ticker"})
    stocks_list[date] = stocks_list['ticker'].map(stocks_request.set_index('ticker')['c'])

    end_time = datetime.now()
    seconds_to_complete = (end_time - start_time).total_seconds()
    times.append(seconds_to_complete)
    iteration = round((np.where(dates==date)[0][0]/len(dates))*100,2)
    iterations_remaining = len(dates) - np.where(dates==date)[0][0]
    average_time_to_complete = np.mean(times)
    estimated_completion_time = (datetime.now() + timedelta(seconds = int(average_time_to_complete*iterations_remaining)))
    time_remaining = estimated_completion_time - datetime.now()
    
    print(f"{iteration}% complete, {time_remaining} left, ETA: {estimated_completion_time}")

0.0% complete, 0:06:03 left, ETA: 2024-08-22 13:55:08.298178
0.4% complete, 0:06:02 left, ETA: 2024-08-22 13:55:08.624403
0.79% complete, 0:06:00 left, ETA: 2024-08-22 13:55:07.882203
1.19% complete, 0:05:59 left, ETA: 2024-08-22 13:55:08.168029
1.58% complete, 0:05:57 left, ETA: 2024-08-22 13:55:07.496120
1.98% complete, 0:05:55 left, ETA: 2024-08-22 13:55:06.764277
2.37% complete, 0:05:54 left, ETA: 2024-08-22 13:55:06.998573
2.77% complete, 0:05:52 left, ETA: 2024-08-22 13:55:06.304001
3.16% complete, 0:05:51 left, ETA: 2024-08-22 13:55:06.644176
3.56% complete, 0:05:49 left, ETA: 2024-08-22 13:55:05.923910
3.95% complete, 0:05:48 left, ETA: 2024-08-22 13:55:06.213474
4.35% complete, 0:05:46 left, ETA: 2024-08-22 13:55:05.473080
4.74% complete, 0:05:45 left, ETA: 2024-08-22 13:55:05.776311
5.14% complete, 0:05:43 left, ETA: 2024-08-22 13:55:05.045498
5.53% complete, 0:05:42 left, ETA: 2024-08-22 13:55:05.319089
5.93% complete, 0:05:40 left, ETA: 2024-08-22 13:55:04.576344
6.32% comp

In [45]:
stocks_list.to_sql("stock_prices", con = engine, if_exists = "replace")

,ticker,2023-07-31,2023-08-01,2023-08-02,2023-08-03,2023-08-04,2023-08-07,2023-08-08,2023-08-09,2023-08-10,...,2024-07-18,2024-07-19,2024-07-22,2024-07-23,2024-07-24,2024-07-25,2024-07-26,2024-07-29,2024-07-30,2024-07-31
0,NVDA,46.729,46.507,44.269,44.515,44.68,45.4165,44.664,42.554,42.388,...,121.09,117.93,123.54,122.59,114.25,112.28,113.06,111.59,103.73,117.02
1,TSLA,267.430,261.070,254.110,259.320,253.86,251.4500,249.700,242.190,245.340,...,249.23,239.20,251.51,246.38,215.99,220.25,219.80,232.10,222.62,232.07
2,AAPL,196.450,195.605,192.580,191.170,181.99,178.8500,179.800,178.190,177.970,...,224.18,224.31,223.96,225.01,218.54,217.49,217.96,218.24,218.80,222.08
3,MSFT,335.920,336.340,327.500,326.660,327.78,330.1100,326.050,322.230,322.930,...,440.37,437.11,442.94,444.85,428.90,418.40,425.27,426.73,422.92,418.35
4,AMD,114.400,117.600,109.350,113.150,115.82,116.8100,113.230,110.470,110.230,...,155.77,151.58,155.87,154.00,144.63,138.32,139.99,139.75,138.44,144.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,APA,40.490,40.240,39.010,41.660,41.87,42.1700,43.330,44.220,44.150,...,32.15,32.01,31.27,30.69,30.28,31.28,31.21,30.78,31.03,31.19
496,LITE,52.360,51.440,50.510,50.960,52.40,52.3600,52.460,50.320,49.910,...,55.50,54.77,55.42,54.80,52.71,50.83,51.83,51.80,49.05,51.78
497,BJ,66.310,66.800,66.770,67.880,68.02,68.6500,68.350,69.100,69.150,...,90.09,89.26,91.21,91.14,89.29,88.37,87.96,87.96,88.52,87.96
498,RF,20.370,20.350,20.260,20.500,20.61,20.9800,20.900,20.610,20.690,...,22.09,22.09,22.39,22.39,22.11,22.55,22.76,22.43,22.73,22.37


In [48]:
stocks_list = stocks_list.set_index('ticker')
stock_returns = stocks_list.pct_change(axis=1)

,2023-07-31,2023-08-01,2023-08-02,2023-08-03,2023-08-04,2023-08-07,2023-08-08,2023-08-09,2023-08-10,2023-08-11,...,2024-07-18,2024-07-19,2024-07-22,2024-07-23,2024-07-24,2024-07-25,2024-07-26,2024-07-29,2024-07-30,2024-07-31
ticker,,,,,,,,,,,,,,,,,,,,,
NVDA,NaN,-0.004751,-0.048122,0.005557,0.003707,0.016484,-0.016569,-0.047242,-0.003901,-0.036166,...,0.026273,-0.026096,0.047571,-0.007690,-0.068032,-0.017243,0.006947,-0.013002,-0.070436,0.128121
TSLA,NaN,-0.023782,-0.026660,0.020503,-0.021055,-0.009493,-0.006960,-0.030076,0.013006,-0.010964,...,0.002938,-0.040244,0.051463,-0.020397,-0.123346,0.019723,-0.002043,0.055960,-0.040844,0.042449
AAPL,NaN,-0.004301,-0.015465,-0.007322,-0.048020,-0.017254,0.005312,-0.008954,-0.001235,-0.001011,...,-0.020535,0.000580,-0.001560,0.004688,-0.028754,-0.004805,0.002161,0.001285,0.002566,0.014991
MSFT,NaN,0.001250,-0.026283,-0.002565,0.003429,0.007108,-0.012299,-0.011716,0.002172,-0.005946,...,-0.007102,-0.007403,0.013338,0.004312,-0.035855,-0.024481,0.016420,0.003433,-0.008928,-0.010806
AMD,NaN,0.027972,-0.070153,0.034751,0.023597,0.008548,-0.030648,-0.024375,-0.002173,-0.024131,...,-0.022957,-0.026899,0.028302,-0.011997,-0.060844,-0.043629,0.012073,-0.001714,-0.009374,0.043629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
APA,NaN,-0.006174,-0.030567,0.067931,0.005041,0.007165,0.027508,0.020540,-0.001583,0.016535,...,-0.002173,-0.004355,-0.023118,-0.018548,-0.013359,0.033025,-0.002238,-0.013778,0.008122,0.005156
LITE,NaN,-0.017571,-0.018079,0.008909,0.028257,-0.000763,0.001910,-0.040793,-0.008148,-0.007614,...,-0.033269,-0.013153,0.011868,-0.011187,-0.038139,-0.035667,0.019673,-0.000579,-0.053089,0.055657
BJ,NaN,0.007390,-0.000449,0.016624,0.002062,0.009262,-0.004370,0.010973,0.000724,0.008677,...,-0.009674,-0.009213,0.021846,-0.000767,-0.020298,-0.010304,-0.004640,0.000000,0.006367,-0.006326


In [54]:
stock_returns = stock_returns.drop(columns=[stock_returns.columns[0]])

In [55]:
stock_returns

,2023-08-01,2023-08-02,2023-08-03,2023-08-04,2023-08-07,2023-08-08,2023-08-09,2023-08-10,2023-08-11,2023-08-14,...,2024-07-18,2024-07-19,2024-07-22,2024-07-23,2024-07-24,2024-07-25,2024-07-26,2024-07-29,2024-07-30,2024-07-31
ticker,,,,,,,,,,,,,,,,,,,,,
NVDA,-0.004751,-0.048122,0.005557,0.003707,0.016484,-0.016569,-0.047242,-0.003901,-0.036166,0.070934,...,0.026273,-0.026096,0.047571,-0.007690,-0.068032,-0.017243,0.006947,-0.013002,-0.070436,0.128121
TSLA,-0.023782,-0.026660,0.020503,-0.021055,-0.009493,-0.006960,-0.030076,0.013006,-0.010964,-0.011910,...,0.002938,-0.040244,0.051463,-0.020397,-0.123346,0.019723,-0.002043,0.055960,-0.040844,0.042449
AAPL,-0.004301,-0.015465,-0.007322,-0.048020,-0.017254,0.005312,-0.008954,-0.001235,-0.001011,0.009393,...,-0.020535,0.000580,-0.001560,0.004688,-0.028754,-0.004805,0.002161,0.001285,0.002566,0.014991
MSFT,0.001250,-0.026283,-0.002565,0.003429,0.007108,-0.012299,-0.011716,0.002172,-0.005946,0.009439,...,-0.007102,-0.007403,0.013338,0.004312,-0.035855,-0.024481,0.016420,0.003433,-0.008928,-0.010806
AMD,0.027972,-0.070153,0.034751,0.023597,0.008548,-0.030648,-0.024375,-0.002173,-0.024131,0.040997,...,-0.022957,-0.026899,0.028302,-0.011997,-0.060844,-0.043629,0.012073,-0.001714,-0.009374,0.043629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
APA,-0.006174,-0.030567,0.067931,0.005041,0.007165,0.027508,0.020540,-0.001583,0.016535,-0.004902,...,-0.002173,-0.004355,-0.023118,-0.018548,-0.013359,0.033025,-0.002238,-0.013778,0.008122,0.005156
LITE,-0.017571,-0.018079,0.008909,0.028257,-0.000763,0.001910,-0.040793,-0.008148,-0.007614,0.021199,...,-0.033269,-0.013153,0.011868,-0.011187,-0.038139,-0.035667,0.019673,-0.000579,-0.053089,0.055657
BJ,0.007390,-0.000449,0.016624,0.002062,0.009262,-0.004370,0.010973,0.000724,0.008677,-0.014624,...,-0.009674,-0.009213,0.021846,-0.000767,-0.020298,-0.010304,-0.004640,0.000000,0.006367,-0.006326


In [ ]:
stock_returns.to_sql("stock_returns", con = engine, if_exists = "replace")